The goal of this notebook is to use the method ```extract_vision_features``` and the datasources in csv to generate ```vision embeddings```.

Note: in this notebook, we only use a sample of 10 patients to make the data processing and extraction easier for our tests.

If ```torchxrayvision``` is not installed, intall it using ``````!pip install torchxrayvision``````

In [ ]:
import torchxrayvision as xrv

import skimage
import cv2
import torch

import torch.nn.functional as F

import pandas as pd
import numpy as np

import os
from os import listdir

import os
os.chdir('../')

from src.data import constants
from src.utils import extract_vision_features

Start by reading sample images and sample patients

In [ ]:
# Read images:
image_path_folder = constants.image_path_folder

# Read sample patients
df_10_dicoms = pd.read_csv(constants.df_10_dicoms)


We start by creating empty dataframes to store the vision embeddings and the concatenation.

Two types of embeddings are generated in this notebook:
- vision dense embeddings
- vision predictions embeddings

For further details, please refer to : https://github.com/mlmed/torchxrayvision/blob/0eafebf36a3f5f30302dff0faaacef5e52243e87/scripts/process_image.py

In [ ]:
#creating empty dataframes to store the vision embeddings and the concatenation:
df_vision_dense_embeddings_fusion = pd.DataFrame()
df_vision_predictions_embeddings_fusion = pd.DataFrame()
vision_embeddings = pd.DataFrame()


#iterating through sample file to read dicom_id for each image and process the corresponding image using torchxrayvision:
for img_id in df_10_dicoms[df_10_dicoms.dicom_id.isin(constants.sample_images)].dicom_id:
    
    for root, dirs, files in os.walk(image_path_folder):
        
        for name in files:
            
            if img_id == name[0:44]: # avoid reading the extension .jpg
                
                # image processing and features extraction:
                img = skimage.io.imread(image_path_folder + name)
                
                # embeddings concatenation for both types:
                df_vision_predictions_embeddings_fusion = df_vision_predictions_embeddings_fusion.append(extract_vision_features(img)[0])
                df_vision_dense_embeddings_fusion = df_vision_dense_embeddings_fusion.append(extract_vision_features(img)[1])
        

# combining both embeddings in one dataframe:
vision_embeddings = pd.concat([ df_vision_predictions_embeddings_fusion , df_vision_dense_embeddings_fusion], axis=1)
vision_embeddings.insert(0, "subject_id", [element for element in df_10_dicoms["subject_id"].unique()])
vision_embeddings.insert(1, "img_id", [element for element in constants.sample_images])

#Display extracted vision_embeddings:
vision_embeddings
